In [1]:
import robotic as ry
import numpy as np
import time

C = ry.Config()

In [2]:
C.addFile("simEnv.g")

# robotPos = C.getFrame("l_robot_base").getPosition()
carPos = C.getFrame("car2").getPosition()
print(carPos)
C.view()

[1.   2.5  0.15]


0

In [3]:
print(C.getFrame('l_robot_base').getPosition())
print(C.getJointState())

[-3.8   0.5   0.01]
[-0.3  0.   0.   0.   0. ]


In [4]:
def moveToTarget(C, target):
    q0 = C.getJointState()

    komo = ry.KOMO(C, 5, 10, 1, False)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e-1])
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0)
    komo.addObjective([], ry.FS.positionDiff, ['l_middle_joint', target.name], ry.OT.eq, [1e1], [0.1, -0.1, 0])
    komo.addObjective([], ry.FS.scalarProductXX, ['l_robot_base', target.name], ry.OT.eq, [1e2], [-1])
    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
    q = komo.getPath()[-1]

    return q

In [5]:
def moveToPark(C, target):
    q0 = C.getJointState()
    if target.getPosition()[0] > 0:
        x = -0.6
        product = 1
    else: 
        x = 0.6
        product = -1

    komo = ry.KOMO(C, 5, 10, 1, False)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e-1])
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0)
    komo.addObjective([], ry.FS.positionDiff, ['car2', target.name], ry.OT.eq, [1e0], [0, -0.1, 0.15])
    komo.addObjective([], ry.FS.scalarProductXX, ['l_robot_base', target.name], ry.OT.eq, [1e2], [product])
    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
    q = komo.getPath()[-1]

    return q

In [6]:
car = C.getFrame('car2')
qHome = C.getJointState()
robot_to_car = moveToTarget(C, car)

In [7]:
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome], [robot_to_car])

#for trial in range(20):
ret = rrt.solve()
path = ret.x
print(ret.feasible)
# if ret.feasible is False:
#     continue
# else:
#     break

-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (l_robot_base)-(car2) [59,51] 	d=-0
True


In [8]:
C.view()
time.sleep(.8)
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

#attach cargo
C.getFrame('car2').unLink()
C.attach('l_robot_base', 'car2') 

target = C.getFrame('parking_space_8')
#robotPos = C.getFrame('l_robot_base').getPosition()
robot_to_park = moveToPark(C, target)

In [9]:
rrt2 = ry.PathFinder()
rrt2.setProblem(C, [robot_to_car], [robot_to_park])

# ret2 = rrt2.solve()
# path2 = ret2.x
for trial in range(10):
    ret2 = rrt2.solve()
    path2 = ret2.x
    print(ret2.feasible)
    if ret2.feasible is False:
        continue
    else:
        break

True


In [10]:
C.view()
time.sleep(.8)
# display the path
for t in range(0, path2.shape[0]-1):
    C.setJointState(path2[t])
    C.view()
    time.sleep(.1)
#attach cargo
C.getFrame('car2').unLink()

In [11]:
return_back = moveToTarget(C, C.getFrame('robot_start'))

rrt3 = ry.PathFinder()
rrt3.setProblem(C, [robot_to_park], [return_back])

ret3 = rrt3.solve()
path3 = ret3.x

C.view()
time.sleep(.8)
# display the path
for t in range(0, path3.shape[0]-1):
    C.setJointState(path3[t])
    C.view()
    time.sleep(.1)

In [12]:
print(C.getFrame("car2").getPosition())
print(C.getFrame("parking_space_28").getPosition())

[ 2.48716081 -4.79165796  0.15      ]
[-0.5  -1.2   0.01]
